In [1]:
#coding=utf-8
import xlwings as xw
import re

# 打開活頁簿檔案
file_path = ('Han-Ggi.xlsx')
wb = xw.Book(file_path)

In [2]:
source_sheet = wb.sheets['jji-im-dui-jau']

end_row = source_sheet.range('A' + str(source_sheet.cells.last_cell.row)).end('up').row
print(f'end_row = {end_row}')

end_row = 4546


In [ ]:
sheet = wb.sheets['jji-im-dui-jau']

row = 1
while row < end_row:
    # Read data from source_sheet 
    zu_im  = str(source_sheet.range('B' + str(row)).value)
    # 沒有注音，取下一個儲存格
    if zu_im.strip() == '':
        row += 1
        continue
    zu_im_chars = len(zu_im)
    
    # ===================================================
    # 處理「聲母」
    # ===================================================
    
    # 測試注音，是否不含聲母
    # pattern = re.compile(r'^[ㄚㆩㄧㆪㄨㆫㄝㆥㄛㆧㄜ儿]')
    pattern = re.compile(r'^[ㄚㆩ(ㄚ。)ㄧㆪ(ㄧ。)ㄨㆫ(ㄨ。)ㄝ(ㄝ。)ㆤㆥㄛㆧ(ㄛ。)ㄜ儿]')
    # 入聲韻尾使用「ㄅ、ㄉ、ㄍ、ㄏ」
    pattern2 = re.compile(r'[ㄅㄉㄍㄏ][-+]$')
    # 轉換成十五音【聲母】
    seng_boo_dict = {
        'ㄅ': 'p', 
        'ㄆ': 'ph', 
        'ㄇ': 'm', 
        'ㆠ': 'b', 
        'ㄉ': 't', 
        'ㄊ': 'th', 
        'ㄋ': 'n', 
        'ㄌ': 'l', 
        'ㄍ': 'k', 
        'ㄎ': 'kh', 
        'ㄏ': 'h', 
        'ㆣ': 'g', 
        'ㄗ': 'ch', 
        'ㄘ': 'chh', 
        'ㄙ': 's', 
        'ㆡ': 'j', 
        'ㄐ': 'ch', 
        'ㄑ': 'chh', 
        'ㄒ': 's', 
        'ㆢ': 'j', 
        'ㄇ': 'm', 
        'ㄣ': 'n', 
        'ㄚ': 'ng',  
    }

    # 轉換成十五音【韻母】
    un_boo_dict = {
        'ㄚ': 'a', 
        'ㄧ': 'i', 
        'ㄨ': 'u', 
        'ㆤ': 'e', 
        'ㄛ': 'oo', 
        'ㄜ': 'o', 
        '儿': 'o', 
        'ㆩ': 'ann', 
        'ㆪ': 'inn', 
        'ㆫ': 'unn', 
        'ㆥ': 'enn', 
        'ㆧ': 'oonn',         
        'ㄢ': 'an',
        'ㄤ': 'ang',
        'ㄣ': 'n',
        'ㄥ': 'ng', 
        'ㄇ': 'm',
        'ㆬ': 'm',
        'ㆭ': 'ng',
    }
    
    # 如果聲母為濁音，則進行聲母符號更換
    im_1 = zu_im[0] 
    im_2 = zu_im[1]
    seng_boo = ''
    if (im_1 == 'ㄅ' and im_2 == '。'):
        seng_boo += 'ㆠ'
        un_boo = zu_im[2:zu_im_chars-1]
    elif (im_1 == 'ㄍ' and im_2 == '。'):
        seng_boo += 'ㆣ'
        un_boo = zu_im[2:zu_im_chars-1]
    elif (im_1 == 'ㄗ' and im_2 == '。'):
        seng_boo += 'ㆡ'
        un_boo = zu_im[2:zu_im_chars-1]
    elif (im_1 == 'ㄐ' and im_2 == '。'):
        seng_boo += 'ㆢ'
        un_boo = zu_im[2:zu_im_chars-1]
    elif (im_1 == 'ㄝ' and im_2 == '。'):
        un_boo = 'ㆥ'
        un_boo += zu_im[2:zu_im_chars-1]
    elif (im_1 == 'ㄝ'):
        un_boo = 'ㆤ'
        un_boo += zu_im[1:zu_im_chars-1]
    else:
        if not pattern.match(zu_im):
            seng_boo += im_1
            un_boo = zu_im[1:zu_im_chars-1]
        else:
            un_boo = zu_im[0:zu_im_chars-1]
            
    # ===================================================
    # 處理「聲調」
    # ===================================================
    seng_tiau = zu_im[zu_im_chars-1]
    buffer = zu_im[zu_im_chars-2:]
    if pattern2.match(buffer):
        if seng_tiau == '-':
            seng_tiau_2 = 8
        elif seng_tiau == '+':
            seng_tiau_2 = 4
    else:
        if seng_tiau == '-':
            seng_tiau_2 = 1
        elif seng_tiau == '+':
            seng_tiau_2 = 7
        elif seng_tiau == 'ˇ':
            seng_tiau_2 = 3
        elif seng_tiau == 'ˊ':
            seng_tiau_2 = 5
        elif seng_tiau == 'ˋ':
            seng_tiau_2 = 2

    # 轉換成十五音
    if seng_boo != '':
        seng = seng_boo_dict[seng_boo]
    
    un_str = un_boo.replace('ㄝ', 'ㆤ')
    un = ''
    last_char = len(un_str) - 1
    i = 0
    for char in un_str:
        if i != last_char:
            un += un_boo_dict[char]
        elif seng_tiau_2 != 4 and seng_tiau_2 != 8:
            un += un_boo_dict[char]
        else:
            un += seng_boo_dict[char]
        i += 1    
    
    # Write to target cell
    sheet.range('C' + str(row)).value = seng_boo
    sheet.range('D' + str(row)).value = un_boo
    sheet.range('E' + str(row)).value = seng_tiau
    sheet.range('F' + str(row)).value = seng
    sheet.range('G' + str(row)).value = un
    sheet.range('H' + str(row)).value = seng_tiau_2
    row += 1
  